<a href="https://colab.research.google.com/github/Mancupfire/Retrieval_Model/blob/main/Retrieval_Model(sBERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.1 MB/s eta 0:00:00


In [ ]:
import json
import numpy as np
import torch
from torch_geometric.data import Data
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load Sentence-BERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Đọc dữ liệu huấn luyện
with open('edinburgh-keywords_train.json', 'r') as f:
    train_data = json.load(f)

keywords = list(train_data['np2count'].keys())

In [ ]:
# Loại bỏ những từ bị trùng
keyword_set = set(keywords)

In [ ]:
def extract_users(info):
    l_user, user2kw = [], []
    for ii in info:
        lus = info[ii]
        for u in lus:
            if u not in l_user:
                l_user.append(u)
                user2kw.append([])
            idx = l_user.index(u)
            user2kw[idx].append(ii)
    return l_user, user2kw

In [ ]:
train_users, train_users2kw = extract_users(train_data['np2users'])

In [ ]:
restaurant_set = set()
listres = []
for kw in train_data['np2rests'].keys():
    listres.extend(train_data['np2rests'][kw].keys())
restaurant_set = set(listres)

In [ ]:
# Tạo ma trận liên kết từ keywords và restaurant
keyword_set = list(keyword_set)
restaurant_set = list(restaurant_set)
restaurants = len(listres)
num_keywords = len(keyword_set)
num_restaurants = len(restaurant_set)
a = np.zeros((num_keywords, num_restaurants))

for kw in train_data['np2rests'].keys():
    for res in train_data['np2rests'][kw].keys():
        idx_kw = keyword_set.index(kw)
        idx_res = restaurant_set.index(res)
        a[idx_kw][idx_res] = 1

In [ ]:
count = 0
for i in listres:
  count += 1
count

124174

In [ ]:
restaurants

124174

In [ ]:
# Mã hóa các từ khóa trong tập huấn luyện
keyword_embeddings = model.encode(list(keyword_set))

In [ ]:
keyword_embeddings

array([[ 0.40866908, -0.03762863,  0.23873019, ..., -0.2974483 ,
         0.1129055 , -0.5735431 ],
       [ 0.6144337 ,  0.5506448 ,  0.36305818, ...,  0.19329049,
         1.1155195 , -0.40919212],
       [ 0.29199535,  0.0910606 , -0.2944026 , ...,  0.22109425,
        -0.45329428, -0.21119182],
       ...,
       [-0.05305937,  0.14181863, -0.41520584, ...,  0.11326814,
        -0.91748834,  0.26066685],
       [-0.07380255,  0.5343566 , -0.3620793 , ..., -0.2756461 ,
        -0.36475393,  0.45706013],
       [-0.5542963 , -0.8777806 ,  0.11086378, ...,  0.4659987 ,
         0.78528965, -0.4688757 ]], dtype=float32)

In [ ]:
# Đọc dữ liệu kiểm tra
with open('edinburgh-keywords_test.json', 'r') as r:
    test_data = json.load(r)

user_keywords = list(test_data['np2reviews'].keys())
user_keywords_list = list(user_keywords)

In [ ]:
test_users, test_users2kw = extract_users(test_data['np2users'])

In [ ]:
# Mã hóa các từ khóa trong tập kiểm tra
test_keywords = [kw for sublist in test_users2kw for kw in sublist]
test_keyword_embeddings = model.encode(test_keywords)

In [ ]:
# Tính toán độ tương đồng giữa từ khóa kiểm tra và từ khóa huấn luyện
similarity_scores = cosine_similarity(test_keyword_embeddings, keyword_embeddings)


In [ ]:
similarity_scores

array([[0.2098643 , 0.21084453, 0.13731132, ..., 0.04635555, 0.08432022,
        0.17438641],
       [0.15400583, 0.14035337, 0.19925159, ..., 0.13843359, 0.14964767,
        0.50328726],
       [0.2796688 , 0.33136547, 0.17913675, ..., 0.24562626, 0.2613269 ,
        0.21795014],
       ...,
       [0.2031357 , 0.07564847, 0.2363351 , ..., 0.30883354, 0.01024354,
        0.06598304],
       [0.10105229, 0.03409922, 0.3079031 , ..., 0.08313295, 0.15013635,
        0.26263592],
       [0.04192811, 0.04776572, 0.2368319 , ..., 0.06451292, 0.05409455,
        0.26175955]], dtype=float32)

In [ ]:
filtered_keywords = []
for i, user_kw in enumerate(test_users2kw):
    updated_user_kw = []
    for kw in user_kw:
        if kw not in keyword_set:
            # Lấy vị trí của test keyword trong ma trận similarity_scores
            test_idx = test_keywords.index(kw)
            sim_scores = similarity_scores[test_idx]

            # Tìm keyword trong traindata có cosine similarity lớn nhất
            best_match_idx = np.argmax(sim_scores)
            best_match_keyword = keyword_set[best_match_idx]

            # Thay thế nhau
            updated_user_kw.append(best_match_keyword)
        else:
            updated_user_kw.append(kw)

In [ ]:
best_match_idx

7796

In [ ]:
# Cập nhật danh sách từ khóa người dùng
filtered_keywords.append(updated_user_kw)

In [ ]:
filtered_keywords

[['chicken',
  'royal mile',
  'cocktail',
  'wife',
  'break',
  'naan',
  'salmon',
  'good food',
  'must',
  4338,
  3286,
  'hake',
  3974,
  5550,
  'class',
  1737,
  6403,
  3697,
  'drawback',
  'english',
  6579,
  1202,
  7796]]

In [ ]:
# Cập nhật lại test_users2kw sau khi thay thế
test_users2kw = filtered_keywords

In [ ]:

for kw in test_users2kw:
    t = np.zeros((1, len(keyword_set)))
    keywords = kw[:10]
    for keys in keywords:
        if keys in keyword_set:
            idx_kw = keyword_set.index(keys)
            t[0][idx_kw] = 1
    R = np.dot(t, a)
    result = np.argsort(R[0])[::-1][:10]

In [ ]:
result

array([853, 220, 682, 666, 820,  22, 548, 844, 613, 826])

In [ ]:
result

array([130,  31, 491, 793, 196, 388, 774, 702, 141, 137])